In [4]:
import datetime

"""시뮬레이션 결과파일에서 데이터 가져오기"""
import sqlite3
import pandas as pd
import win32com.client  # Windows COM 인터페이스 사용
import os
import xml.etree.ElementTree as ET

# 폴더 경로 설정
folder_path = r"C:\VISSIM_Workspace\test3"  # 폴더 경로 입력 받아야함 추후에 함수에 파라미터로 처리 예정
# 폴더 내 파일 목록 가져오기
file_list = os.listdir(folder_path)
inpx_files = [file for file in file_list if file.endswith(".inpx")]

for i in range(len(inpx_files)):
    file_name = inpx_files[i]
    print("file_name : ", file_name)
    file_path = folder_path + "\\" +file_name

    # VISSIM 실행 및 연결
    Vissim = win32com.client.Dispatch("Vissim.Vissim")  # Vissim 인스턴스 생성

    # VISSIM 네트워크 파일 (.inpx) 로드
    # C:\VISSIM_Workspace\test\test.inpx"

    Vissim.LoadNet(file_path)

    print("VISSIM 시뮬레이션 실행 중...")
    Vissim.Simulation.RunContinuous()

    Vissim.Simulation.Stop()
    print("VISSIM 시뮬레이션 종료")

    Vissim.exit()

    # XML 파일 파싱
    tree = ET.parse(file_path)
    root = tree.getroot()
    interval_value = 0
    # dataColl 태그에서 interval 속성 값 찾기
    for elem in root.iter("dataColl"):
        interval_value = int(elem.get("interval"))  # "interval" 속성 값 가져오기
        if interval_value:
            print(f"추출된 interval 값: {interval_value}") # 60
        else:
            print("추출된 interval 값이 없습니다.")

    # 데이터베이스 파일 경로
    # "C:/VISSIM_Workspace/test/test.results/3.db"
    # 확장자 제거
    results_path = folder_path + "\\" +os.path.splitext(file_name)[0] + ".results"
    results_file = [f for f in os.listdir(results_path) if f.endswith(".db")][-1]

    db_path = os.path.join(folder_path, os.path.splitext(file_name)[0] + ".results",  results_file)
    # 파일명 추출
    simrun = db_path.split("\\")[-1].split(".")[0]


    # 데이터베이스 연결
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 특정 테이블의 데이터 조회
    table_name = "DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass"
    query = f"SELECT * FROM {table_name};"  # 전체 데이터 조회

    # 데이터 읽기
    df = pd.read_sql_query(query, conn)

    # TIMEINT 변환
    df["ARG_TIMEINTERVAL"] = df["ARG_TIMEINTERVAL"].astype(int) * interval_value - interval_value  # 변환 시작점 보정
    df["ARG_TIMEINTERVAL"] = df["ARG_TIMEINTERVAL"].apply(lambda x: f"{x}-{x+interval_value}")

    # OCCUPRATE 보정
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(float)*100
    df["OCCUPRATE"] = df["OCCUPRATE"].apply(lambda x: f"{x:.2f}")  # 항상 소수점 둘째 자리까지 유지
    df = df.round(2)
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(str) + "%"

    # SIMRUN 설정
    df["SIMRUN"] =simrun

    # 컬럼명 변경
    df.rename(columns={
        "ARG_TIMEINTERVAL": "TIMEINT",
        "OBJECT_ID": "DATACOLLECTIONMEASUREMENT",
        "ACCELERATION" : "ACCELERATION(ALL)",
        "DIST" : "DIST(ALL)",
        "LENGTH" : "LENGTH(ALL)",
        "VEHS" : "VEHS(ALL)",
        "PERS" : "PERS(ALL)",
        "QUEUEDELAY" : "QUEUEDELAY(ALL)",
        "SPEEDAVGARITH" : "SPEEDAVGARITH(ALL)",
        "SPEEDAVGHARM" : "SPEEDAVGHARM(ALL)",
        "OCCUPRATE" : "OCCUPRATE(ALL)"
    }, inplace=True)

    # 순서 재정의
    df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
             "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)"]]

    excel_dr  = os.path.join(folder_path + "\\" + os.path.splitext(file_name)[0] + "_output")
    results_file = os.path.splitext(results_file)[0]
    excel_path = excel_dr +"\\" + results_file + "_output" +".xlsx"
    if not os.path.exists(excel_dr):
        os.makedirs(excel_dr)
    df.to_excel(excel_path, index=False)

    # 연결 종료
    conn.close()


file_name :  test.inpx
VISSIM 시뮬레이션 실행 중...
VISSIM 시뮬레이션 종료
추출된 interval 값: 60
file_name :  test2.inpx
VISSIM 시뮬레이션 실행 중...
VISSIM 시뮬레이션 종료
추출된 interval 값: 60
